In [1]:
import pandas as pd
pd.options.plotting.backend = "plotly"
from datetime import datetime, date
import pylatex as pl
from pylatex.utils import NoEscape
import pymc as pm
import arviz as az
import nevergrad as ng
import plotly.express as px

from estival.model import BayesianCompartmentalModel
from estival.optimization.nevergrad import optimize_model
from estival.priors import UniformPrior
from estival.targets import NegativeBinomialTarget, CustomTarget
from estival.calibration import pymc as epm
from tbdynamics import model
from tbdynamics.inputs import *

WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
time_start = 1800
time_end = 2020
time_step = 0.1

In [3]:
compartments = [
    "susceptible",
    "early_latent",
    "late_latent",
    "infectious",
    "on_treatment",
    "recovered",
]
infectious_compartments = [
    "infectious",
    "on_treatment",
]

latent_compartments = [
    "early_latent",
    "late_latent",
]

In [4]:
params = {
    "contact_rate": 0.009414102898074345,
    "start_population_size": 280000,
    "progression_multiplier": 1.1,
    "infectious_seed": 1,
    "rr_infection_latent": 0.2,
    "rr_infection_recovered": 0.2,
    "infect_death_rate_unstratified": 0.21,
    "on_treatment_infect_multiplier": 0.4,
}

In [5]:
tb_model, build_text = model.build_base_model(compartments, infectious_compartments, time_start, time_end, time_step)

In [6]:
model.set_starting_conditions(tb_model)


'The simulation starts with Parameter start_population_size million fully susceptible persons, with infectious persons introduced later through strain seeding as described below. '

In [7]:
model.add_entry_flow(tb_model)

Function: 'scaled_curve', args=(ModelVariable time, Function: 'ge...,), kwargs={})), kwargs={})


'The birth process add newborns to the model'

In [8]:
model.add_natural_death_flow(tb_model)

In [9]:
model.add_infection(tb_model)

('The infection process moves people from the susceptible compartment to the early_latent compartment, under the frequency-dependent transmission assumption. ',
 'The infection_from_latent process moves people from the late_latent compartment to the early_latent compartment, under the frequency-dependent transmission assumption. ',
 'The infection_from_recovered process moves people from the recovered compartment to the early_latent compartment, under the frequency-dependent transmission assumption. ')

In [10]:
model.add_latency(tb_model)

('The stabilisation process moves people from the early_latent compartment to the late_latent compartment, under the frequency-dependent transmission assumption. ',
 'The early_activation process moves people from the early_latent compartment to the infectious compartment, under the frequency-dependent transmission assumption. ',
 'The late_activation process moves people from the late_latent compartment to the infectious compartment, under the frequency-dependent transmission assumption. ')

In [11]:
model.add_infect_death(tb_model)

In [12]:
model.add_detection(tb_model)

'The detection process moves people from the infectious compartment to the on_treatment compartment, under the frequency-dependent transmission assumption. '

In [13]:
model.add_treatment_related_outcomes(tb_model)

('The treatment_recovery process moves people from the on_treatment compartment to the recovered compartment, under the frequency-dependent transmission assumption. ',
 'The treatment_death process moves people from the on_treatment compartment to the death, under the frequency-dependent transmission assumption. ',
 'The early_activation process moves people from the on_treatment compartment to the infectious compartment, under the frequency-dependent transmission assumption. ')

In [14]:
age_strata = [0,5,15,35,50]

In [15]:
matrix = model.build_contact_matrix()

In [16]:
strat = model.add_age_strat(compartments, infectious_compartments, [0,5,15,35,50], matrix, fixed_parameters)

In [17]:
tb_model.stratify_with(strat)

In [18]:
model.request_output(tb_model, compartments,  latent_compartments, infectious_compartments)

In [19]:
tb_model.run(parameters=params)

In [21]:
tb_model.get_outputs_df()

,susceptibleXage_0,susceptibleXage_5,susceptibleXage_15,susceptibleXage_35,susceptibleXage_50,early_latentXage_0,early_latentXage_5,early_latentXage_15,early_latentXage_35,early_latentXage_50,...,on_treatmentXage_0,on_treatmentXage_5,on_treatmentXage_15,on_treatmentXage_35,on_treatmentXage_50,recoveredXage_0,recoveredXage_5,recoveredXage_15,recoveredXage_35,recoveredXage_50
1800.0,55999.800000,55999.800000,55999.800000,55999.800000,55999.800000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1800.1,55722.900537,56522.426390,56264.203225,55854.599090,55581.400175,0.057901,0.118947,0.386481,0.464600,0.547165,...,0.018218,0.018768,0.018526,0.018406,0.018360,0.000185,0.000192,0.000190,0.000189,0.000188
1800.2,55452.623154,57034.123760,56532.385310,55711.852176,55167.969592,0.082575,0.183136,0.681718,0.816363,0.956994,...,0.033854,0.035859,0.034600,0.034190,0.034068,0.000700,0.000744,0.000727,0.000718,0.000713
1800.3,55188.829660,57535.133556,56804.207960,55571.546017,54759.438610,0.093457,0.218949,0.912345,1.088676,1.270655,...,0.047970,0.052096,0.048836,0.048042,0.047878,0.001493,0.001634,0.001570,0.001540,0.001527
1800.4,54931.385489,58025.693007,57079.538677,55433.670319,54355.742861,0.097862,0.239846,1.097509,1.305172,1.517086,...,0.060943,0.067859,0.061701,0.060469,0.060331,0.002535,0.002850,0.002688,0.002622,0.002596
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019.6,353911.747406,618616.197210,460809.062264,115369.790467,31736.053527,269.686433,1130.638503,20875.032017,13799.712680,9499.182161,...,653.466327,2368.569193,5737.043525,4190.623524,3124.814606,1652.420853,24421.238543,105588.933369,81985.233560,58435.483516
2019.7,353933.265056,618939.435804,461674.323067,115744.868945,31878.376426,269.555967,1125.060055,21034.776345,13818.291260,9516.388320,...,654.004487,2361.013286,5730.918086,4192.869530,3129.475594,1647.958381,24303.265352,105385.666503,81972.628113,58514.041979
2019.8,353955.802924,619260.724974,462511.100900,116117.763913,32020.314743,269.092183,1120.771875,21186.136769,13837.615905,9533.710399,...,654.641577,2353.457587,5725.106911,4195.099168,3134.118093,1643.614498,24186.110481,105182.299996,81959.199629,58591.882965
2019.9,353980.115660,619579.102849,463337.750099,116488.367177,32161.843993,268.014502,1118.380583,21314.990076,13857.572910,9551.184862,...,655.370973,2345.937555,5719.693666,4197.317943,3138.741212,1639.388488,24069.764136,104978.847054,81944.945391,58669.001587
